In [1]:
import numpy as np
import tensorflow as tf
import random

C:\Users\xnive\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\xnive\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\xnive\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


# Getting more than linear estimations by training our weights

The idea of this project is to create an architecture that is small and is able to detect non-linear relationships between data. We will first explain this architecture and then make some experiments.

The idea behind this architecture is to have a Neural Network, named Task, with inputs I, outputs O and weights W, and instead of regular backpropagation to learn the weights W we implement a second Neural network, named Lift, with inputs I and outputs W (approximations of the weights of the first NN). For training we will consider the "right" input to be the weights after a pass of backpropragation, and use MSE to train the network Lift.

For this task we will use a simple MLP network for both of our networks but it should be able to extend to any architecture in a trivial way.

## Task

In [2]:
def task():
    task=tf.keras.Sequential(layers=[tf.keras.layers.Dense(5, activation="relu", input_shape=(3,)), tf.keras.layers.Dense(1)])
    task.compile(loss="mse")
    return task

## Lift

In [3]:
def lift():
    lift=tf.keras.Sequential(layers=[tf.keras.layers.Dense(10), tf.keras.layers.Dense(26)])
    lift.compile(loss="mse")
    return lift

## Model

In [17]:
class lifter(tf.keras.Model):
    def __init__(self):
        super(lifter, self).__init__()
    
    def build(self):
        self.task=tf.keras.Sequential(layers=[tf.keras.layers.Dense(5, activation="relu", input_shape=(3,)), tf.keras.layers.Dense(1)])
        self.lift=tf.keras.Sequential(layers=[tf.keras.layers.Dense(10, activation="relu", input_shape=(3,)), tf.keras.layers.Dense(26)])
        self.task.compile(loss="mse")
        self.lift.compile(loss="mse")
        weights=self.task.get_weights()
        self.weight_shapes=[np.shape(weight) for weight in weights]

    def call(self, X):
        new_weights=self.lift(X).numpy()
        j=0
        start=0
        end=0
        end2=0
        for layer in self.task.layers:
            end+=get_size(self.weight_shapes[j])
            end2=end+get_size(self.weight_shapes[j+1])
            new_layer_weights=[np.reshape(new_weights[:,start:end], self.weight_shapes[j]), np.reshape(new_weights[:,end:end2], self.weight_shapes[j+1])]
            layer.set_weights(new_layer_weights)
            start=end2
            end=end2
            j+=2
        return self.task(X)

    def fit(self, X,y, epochs=1, print_every=1000):
        for i in range(len(X)):
            if (i+1)%print_every==0:
                print(i+1)
            for j in range(epochs):
                self.call(X[i:i+1])
                self.task.fit(X[i:i+1],y[i:i+1], verbose=0)
                weights=flatten_weights(self.task.get_weights())
                self.lift.fit(X[i:i+1], weights, verbose=0)
        

# Data and Training

We will make a small artificial task where the data has inputs (x,y,z) and outputs x^2+2yz

In [5]:
random.seed(0)
X=np.array([[random.randrange(0,10), random.randrange(0,10), random.randrange(0,10)] for i in range(10000)])
y=np.array([X[i,0]**2+2*X[i,1]*X[i,2] for i in range(10000)])

X_test=np.array([[random.randrange(0,10), random.randrange(0,10), random.randrange(0,10)] for i in range(10000)])
y_test=np.array([X_test[i,0]**2+2*X_test[i,1]*X_test[i,2] for i in range(10000)])

In [6]:
task=tf.keras.Sequential(layers=[tf.keras.layers.Dense(5, activation="relu", input_shape=(3,)), tf.keras.layers.Dense(1)])
lift=tf.keras.Sequential(layers=[tf.keras.layers.Dense(10, activation="relu", input_shape=(3,)), tf.keras.layers.Dense(26)])
task.compile(loss="mse")
lift.compile(loss="mse")

In [9]:
def flatten_weights(weights):
    flat=np.array([])
    for weight in weights:
        flat=np.concatenate([flat, weight.flatten()])
    return np.array([flat])

In [10]:
def get_size(shape):
    size=1
    for i in shape:
        size*=i
    return size

In [9]:
weights=task.get_weights()
weight_shapes=[np.shape(weight) for weight in weights]
weights

[array([[ 0.5739555 , -0.40978923,  0.10870063,  0.62186915,  0.5207469 ],
        [ 0.22630471, -0.4845951 ,  0.46104378, -0.33550537,  0.61389977],
        [ 0.66520506,  0.43206006,  0.6068682 , -0.48597577,  0.77198094]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([[-0.30677247],
        [ 0.750705  ],
        [ 0.3262534 ],
        [-0.6196914 ],
        [-0.5059142 ]], dtype=float32),
 array([0.], dtype=float32)]

In [10]:
weight_shapes

[(3, 5), (5,), (5, 1), (1,)]

In [11]:
flatten_weights(weights)

array([[ 0.57395548, -0.40978923,  0.10870063,  0.62186915,  0.52074689,
         0.22630471, -0.48459509,  0.46104378, -0.33550537,  0.61389977,
         0.66520506,  0.43206006,  0.60686821, -0.48597577,  0.77198094,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.30677247,  0.750705  ,  0.32625341, -0.61969137, -0.50591421,
         0.        ]])

In [13]:
for i in range(10000):
    if i+1%500 == 0:
        print(i)
    task.fit(X[i:i+1],y[i:i+1], verbose=0)
    weights=flatten_weights(task.get_weights())
    lift.fit(X[i:i+1], weights, verbose=0)
    new_weights=lift(X[i+1:i+2]).numpy()
    j=0
    start=0
    end=0
    end2=0
    for layer in task.layers:
        end+=get_size(weight_shapes[j])
        end2=end+get_size(weight_shapes[j+1])
        new_layer_weights=[np.reshape(new_weights[:,start:end], weight_shapes[j]), np.reshape(new_weights[:,end:end2], weight_shapes[j+1])]
        layer.set_weights(new_layer_weights)
        start=end2
        end=end2
        j+=2

KeyboardInterrupt: 

In [27]:
new_weights=lift(np.array([[2,4,0]])).numpy()
j=0
start=0
end=0
end2=0
for layer in task.layers:
        end+=get_size(weight_shapes[j])
        end2=end+get_size(weight_shapes[j+1])
        new_layer_weights=[np.reshape(new_weights[:,start:end], weight_shapes[j]), np.reshape(new_weights[:,end:end2], weight_shapes[j+1])]
        layer.set_weights(new_layer_weights)
        start=end2
        end=end2
        j+=2
task(np.array([[2,4,0]])).numpy()

array([[6.732047]], dtype=float32)

In [22]:
model=lifter()
model.build()
model.fit(X, y, epochs=1)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [28]:
model.call(np.array([[7,5,2]]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[67.78032]], dtype=float32)>